In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%run assignment1_Kaustav.ipynb

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def compute_cost(X, y, theta):
    m = len(y)
    h = sigmoid(X @ theta)
    epsilon = 1e-5
    cost = (1/m)*(((-y).T @ np.log(h + epsilon))-((1-y).T @ np.log(1-h + epsilon)))
    return cost

In [ ]:
def gradient_descent(X, y, params, learning_rate, iterations):
    m = len(y)
    cost_history = np.zeros((iterations,1))

    for i in range(iterations):
        params = params - (learning_rate/m) * (X.T @ (sigmoid(X @ params) - y)) 
        cost_history[i] = compute_cost(X, y, params)

    return (cost_history, params)

In [ ]:
def predict(X, params):
    return np.round(sigmoid(X @ params))

In [ ]:
def kFoldCrossVal(k, X, y):
    accuracy = 0
    # model = NaiveBayes()
    size = (int)(len(y)/k)
    for i in range(k):
        train_x = X[np.r_[0:size*i, size*(i+1):]]
        train_y = y[np.r_[0:size*i, size*(i+1):]]
        test_x = X[(i*size):size*(i+1)]
        test_y = y[(i*size):size*(i+1)]
        
        model.fit(train_x, train_y)
        predictions = model.predict(test_x)
        run_accuracy = model.evaluate_acc(test_y, predictions)
        accuracy = accuracy + run_accuracy
        print(run_accuracy)

    return accuracy / k

In [ ]:
def evaluate_acc(self, test_y, predicted_y):
        return (np.sum(predicted_y == test_y) / len(test_y))

In [ ]:
df.head(10)

features = df.drop(['salary'] , axis=1)
X = features.values
output = df['salary']
y = output.values

y = y[:,np.newaxis]
sns.set_style('white')
sns.scatterplot(X[:,0],X[:,1],hue=y.reshape(-1));

m = len(y)

X = np.hstack((np.ones((m,1)),X))
n = np.size(X,1)
params = np.zeros((n,1))

iterations = 5
learning_rate = 0.001

initial_cost = compute_cost(X, y, params)

print("Initial Cost is: {} \n".format(initial_cost))

(cost_history, params_optimal) = gradient_descent(X, y, params, learning_rate, iterations)

print("Optimal Parameters are: \n", params_optimal, "\n")

plt.figure()
sns.set_style('white')
plt.plot(range(len(cost_history)), cost_history, 'r')
plt.title("Convergence Graph of Cost Function")
plt.xlabel("Number of Iterations")
plt.ylabel("Cost")
plt.show()

y_pred = predict(X, params_optimal)
score = float(sum(y_pred == y))/ float(len(y))

print(score)
